In [1]:
import time
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from pandas.plotting import parallel_coordinates
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler # 표준화(평균을 기준)
from sklearn.preprocessing import MinMaxScaler # 정규화(정해진 규칙 기준, mix-max기준)
from sklearn.datasets import load_iris
# cross_val_score: 다양한 샘플링으 ㄹ해서 여러번 테스트 --> 평균치를 구해본다.!
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder


%matplotlib inline

# 주요 라이브러리 import

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm


# 디렉토리 및 파일 이름에 맞추어 변경
font_location = 'C:/Users/hi/Himedia_jupyter/NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)
import scipy.io as spio

from pandas.plotting import parallel_coordinates

In [13]:
df = pd.read_csv('../csv-data/predictive_maintenance.csv')
df.set_index('UDI', inplace=True)
df.isna().sum()

Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Target                     0
Failure Type               0
dtype: int64

In [14]:
df

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
UDI,,,,,,,,,
1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...
9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure


In [15]:
df = df.drop(columns=['Product ID', 'Type', 'Process temperature [K]'])
df['Failure Type'] = df['Failure Type'].astype('category')
df1 = pd.get_dummies(df)
df1

,Air temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type_Heat Dissipation Failure,Failure Type_No Failure,Failure Type_Overstrain Failure,Failure Type_Power Failure,Failure Type_Random Failures,Failure Type_Tool Wear Failure
UDI,,,,,,,,,,,
1,298.1,1551,42.8,0,0,0,1,0,0,0,0
2,298.2,1408,46.3,3,0,0,1,0,0,0,0
3,298.1,1498,49.4,5,0,0,1,0,0,0,0
4,298.2,1433,39.5,7,0,0,1,0,0,0,0
5,298.2,1408,40.0,9,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9996,298.8,1604,29.5,14,0,0,1,0,0,0,0
9997,298.9,1632,31.8,17,0,0,1,0,0,0,0
9998,299.0,1645,33.4,22,0,0,1,0,0,0,0


In [28]:
df = df1

In [29]:
df.corr()

,Air temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type_Heat Dissipation Failure,Failure Type_No Failure,Failure Type_Overstrain Failure,Failure Type_Power Failure,Failure Type_Random Failures,Failure Type_Tool Wear Failure
Air temperature [K],1.000000,0.022670,-0.013778,0.013853,0.082556,0.136372,-0.084453,-0.006072,0.003470,0.016172,0.009545
Rotational speed [rpm],0.022670,1.000000,-0.875027,0.000223,-0.044188,-0.119213,0.045483,-0.091264,0.123018,-0.011685,0.011960
Torque [Nm],-0.013778,-0.875027,1.000000,-0.003093,0.191321,0.136570,-0.191563,0.150239,0.083781,0.015060,-0.018617
Tool wear [min],0.013853,0.000223,-0.003093,1.000000,0.105448,-0.001023,-0.105251,0.139669,-0.009334,0.007964,0.114717
Target,0.082556,-0.044188,0.191321,0.105448,1.000000,0.568154,-0.959389,0.473324,0.522812,-0.007955,0.358919
Failure Type_Heat Dissipation Failure,0.136372,-0.119213,0.136570,-0.001023,0.568154,1.000000,-0.560498,-0.009436,-0.010423,-0.004519,-0.007156
Failure Type_No Failure,-0.084453,0.045483,-0.191563,-0.105251,-0.959389,-0.560498,1.000000,-0.466946,-0.515767,-0.223638,-0.354083
Failure Type_Overstrain Failure,-0.006072,-0.091264,0.150239,0.139669,0.473324,-0.009436,-0.466946,1.000000,-0.008683,-0.003765,-0.005961
Failure Type_Power Failure,0.003470,0.123018,0.083781,-0.009334,0.522812,-0.010423,-0.515767,-0.008683,1.000000,-0.004159,-0.006584
Failure Type_Random Failures,0.016172,-0.011685,0.015060,0.007964,-0.007955,-0.004519,-0.223638,-0.003765,-0.004159,1.000000,-0.002855


In [30]:
df.columns

Index(['Air temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Target', 'Failure Type_Heat Dissipation Failure',
       'Failure Type_No Failure', 'Failure Type_Overstrain Failure',
       'Failure Type_Power Failure', 'Failure Type_Random Failures',
       'Failure Type_Tool Wear Failure'],
      dtype='object')

In [31]:
df['Target'].value_counts()

0    9661
1     339
Name: Target, dtype: int64

In [22]:
# df['Type'] = df['Type'].astype('category')
# df['Failure Type'] = df['Failure Type'].astype('category')
# df.info()

In [23]:
df.isna().sum()

Air temperature [K]                      0
Rotational speed [rpm]                   0
Torque [Nm]                              0
Tool wear [min]                          0
Target                                   0
Failure Type_Heat Dissipation Failure    0
Failure Type_No Failure                  0
Failure Type_Overstrain Failure          0
Failure Type_Power Failure               0
Failure Type_Random Failures             0
Failure Type_Tool Wear Failure           0
dtype: int64

In [401]:
df.columns

Index(['Air temperature [K]', 'Torque [Nm]', 'Tool wear [min]', 'Target'], dtype='object')

In [42]:
data = ['Air temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]','Tool wear [min]', 'Failure Type_Heat Dissipation Failure','Failure Type_No Failure', 'Failure Type_Overstrain Failure','Failure Type_Power Failure', 'Failure Type_Random Failures','Failure Type_Tool Wear Failure']
target = 'Target'

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    df[data],
    df[target],
    test_size=0.2,
    random_state=42,
    stratify=df['Target']
)

In [44]:
## shape 확인
print('----Shape 확인----')
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test', y_test.shape)

----Shape 확인----
X_train: (8000, 10)
y_train: (8000,)
X_test: (2000, 10)
y_test (2000,)


In [45]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 4059 to 1425
Data columns (total 10 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Air temperature [K]                    8000 non-null   float64
 1   Rotational speed [rpm]                 8000 non-null   int64  
 2   Torque [Nm]                            8000 non-null   float64
 3   Tool wear [min]                        8000 non-null   int64  
 4   Failure Type_Heat Dissipation Failure  8000 non-null   uint8  
 5   Failure Type_No Failure                8000 non-null   uint8  
 6   Failure Type_Overstrain Failure        8000 non-null   uint8  
 7   Failure Type_Power Failure             8000 non-null   uint8  
 8   Failure Type_Random Failures           8000 non-null   uint8  
 9   Failure Type_Tool Wear Failure         8000 non-null   uint8  
dtypes: float64(2), int64(2), uint8(6)
memory usage: 359.4 KB


In [48]:
model_list = []
LR = LogisticRegression()
NB = GaussianNB()
RF = RandomForestClassifier()
SVC = SVC
Dtree = DecisionTreeClassifier()
XGB = XGBClassifier()
KNN = KNeighborsClassifier()
model_list.append(("LR",LR))
model_list.append(("NB",NB))
model_list.append(("RF",RF))
model_list.append(("SVC",SVC))
model_list.append(("Dtree",Dtree))
model_list.append(("XGB",XGB))
model_list.append(("KNN",KNN))
model_list

[('LR', LogisticRegression()),
 ('NB', GaussianNB()),
 ('RF', RandomForestClassifier()),
 ('SVC', SVC()),
 ('Dtree', DecisionTreeClassifier()),
 ('XGB',
  XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
                colsample_bynode=None, colsample_bytree=None,
                enable_categorical=False, gamma=None, gpu_id=None,
                importance_type=None, interaction_constraints=None,
                learning_rate=None, max_delta_step=None, max_depth=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=None, num_parallel_tree=None,
                predictor=None, random_state=None, reg_alpha=None,
                reg_lambda=None, scale_pos_weight=None, subsample=None,
                tree_method=None, validate_parameters=None, verbosity=None)),
 ('KNN', KNeighborsClassifier())]

In [51]:
def print_cv(model, X_train, y_train, k):
    scores = cross_val_score(model, X_train, y_train, cv = k)
    print(i, k, '----교차검증시작----')
    print('결과:', scores)
    mean_scores = np.mean(scores)
    print('평균:', mean_scores)
    return scores, mean_scores 
# 함수의 결과로 여러개의 데이터를 return
# 튜플로 묶어서 반환하게 된다.(scores, mean_scores)

In [52]:
k_number=[5, 10]
sc_list = []
mean_sc_list = []
for i, model in model_list:
    for k in k_number:
        sc, mean_sc = print_cv(model, X_train, y_train, k)
        sc_list.append(sc)
        mean_sc_list.append(mean_sc)
    print()
print('**********************************************************************************************************************')
print()
print()
print('모델명:', model_list,'k:', k_number)
print()
print()
print('전체 모델의 교차 검증 결과 리스트:')
print(sc_list)
print('전체 모델의 교차 검증 결과 평균:')
print(mean_sc_list)
print()
# 최대 평균값 ==
mean_sc_list_nan = []
for i in mean_sc_list:
    if np.isnan(i):
        pass
    else:
        mean_sc_list_nan.append(i)
max_value = np.max(mean_sc_list_nan)
# 최대 평균값 인덱스 ==


max_index = np.nanargmax(mean_sc_list)
# 최대 평균값 모델명 ==
max_model = model_list[max_index//2][0]
if max_index%2 ==0:
    k_max = 5
else:
    k_max = 10

print('최고 교차 검증 평균>> ', max_value)
print('최고 교차 검증 인덱스>> ', max_index)
print('최고 교차 검증 모델링>> ', max_model, 'k =', k_max)



LR 5 ----교차검증시작----
결과: [0.99875  0.998125 0.999375 0.999375 1.      ]
평균: 0.999125
LR 10 ----교차검증시작----
결과: [1.      0.9975  1.      0.99625 1.      0.99875 1.      0.99875 1.
 1.     ]
평균: 0.999125

NB 5 ----교차검증시작----
결과: [0.99875  0.998125 0.999375 0.999375 1.      ]
평균: 0.999125
NB 10 ----교차검증시작----
결과: [1.      0.9975  1.      0.99625 1.      0.99875 1.      0.99875 1.
 1.     ]
평균: 0.999125

RF 5 ----교차검증시작----
결과: [0.99875  0.998125 0.999375 0.999375 1.      ]
평균: 0.999125
RF 10 ----교차검증시작----
결과: [1.      0.9975  1.      0.99625 1.      0.99875 1.      0.99875 1.
 1.     ]
평균: 0.999125

SVC 5 ----교차검증시작----
결과: [0.96625  0.96625  0.96625  0.96625  0.965625]
평균: 0.9661250000000001
SVC 10 ----교차검증시작----
결과: [0.96625 0.96625 0.96625 0.96625 0.96625 0.96625 0.96625 0.96625 0.96625
 0.965  ]
평균: 0.9661250000000001

Dtree 5 ----교차검증시작----
결과: [0.998125 0.996875 0.996875 0.998125 0.99875 ]
평균: 0.9977500000000001
Dtree 10 ----교차검증시작----
결과: [0.9975  0.9975  1.      0.99625 0.99875 0.9

In [53]:
mean_sc_list

[0.999125,
 0.999125,
 0.999125,
 0.999125,
 0.999125,
 0.999125,
 0.9661250000000001,
 0.9661250000000001,
 0.9977500000000001,
 0.9981249999999999,
 nan,
 nan,
 0.9702500000000001,
 0.9698749999999998]

In [54]:
def grid_search(model, params, k):
    gs_model = GridSearchCV(model, params, cv=k)
    ## gs_model는 최적의 파라메터가 적요된 모델의 객체!
    gs_model.fit(X_train, y_train)
    print('best_estimator:',gs_model.best_estimator_) ##최적의 파라메터가 적용된 모델 객체
    print('best_score:',gs_model.best_score_) ##최적의 파라메터가 적용된 모델로 학습한 정확도 결과(acc)
    print('best_params:',gs_model.best_params_) ##최적의 파라메터
    print('gs_model:',gs_model) #grid_search 객체

In [55]:
params = { #하이퍼파라메터 설정으로 가질 수 있는 모든 경우의 수=> 4*3*2: 24
    'max_depth':[3, 5, 7, 10], #key:하이퍼파라메터명, values는 list
    'random_state': [1,3,5],
    'criterion': ['gini','entropy'] #혼란스러운 정도(분류의 정도)를 계산하는 기준
}

for i, model in model_list:
    try:
        print('----clf model:', model, '----')
        grid_search(model, params, 10)
        print()
    except Exception as e:
        print(e)
        print()

----clf model: LogisticRegression() ----
Invalid parameter criterion for estimator LogisticRegression(). Check the list of available parameters with `estimator.get_params().keys()`.

----clf model: GaussianNB() ----
Invalid parameter criterion for estimator GaussianNB(). Check the list of available parameters with `estimator.get_params().keys()`.

----clf model: RandomForestClassifier() ----
best_estimator: RandomForestClassifier(max_depth=3, random_state=1)
best_score: 0.999125
best_params: {'criterion': 'gini', 'max_depth': 3, 'random_state': 1}
gs_model: GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 7, 10],
                         'random_state': [1, 3, 5]})

----clf model: SVC() ----
Invalid parameter criterion for estimator SVC(). Check the list of available parameters with `estimator.get_params().keys()`.

----clf model: DecisionTreeClassifier() ----
best_estimator: 

In [56]:
KNN.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [58]:
GaussianNB().get_params().keys()

dict_keys(['priors', 'var_smoothing'])

In [59]:
LogisticRegression().get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [63]:
## 최적의 하이퍼 파라메터로 모델 객체 생성
## fit()
dt_gs = DecisionTreeClassifier(criterion = 'gini', max_depth=3, random_state=1)
dt_gs.fit(X_train, y_train)
rf_gs = RandomForestClassifier(criterion = 'gini', max_depth=3, random_state=1)
rf_gs.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, random_state=1)

In [64]:
for i , model in model_list:
    print(i)
    print(model)

LR
LogisticRegression()
NB
GaussianNB()
RF
RandomForestClassifier()
SVC
SVC()
Dtree
DecisionTreeClassifier()
XGB
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
KNN
KNeighborsClassifier()


In [65]:
## 테스트 데이터로 검증해보자!
def test_validation(model, X_test, y_test):
    predict_result = model.predict(X_test)
    acc = accuracy_score(y_test, predict_result)
    print("accuracy:", acc)
    cf = confusion_matrix(y_test, predict_result)
    ##실제(c) --> 예측(c, sg), 실제(sg)==> 예측(sg,c),
    # 이진분류(binary classification)인 경우에는
    # 예측할 수 있는 경우의 수는 4가지!
    # 4가지를 표형태로 정리해주는 것이 confusion matrix!
    # 첫반째 행/열은 c, 두번째 행/열은 sg, abc순으로 행/열이 결정
    print('confusion_matrix:\n', cf)
    ## confusion matrix는 binary classfication 인 경우에는 보기가 괜찮으나, 
    ## multi class classification인 경우에는 보기가 힘들어서
    ## classification_report 를 사용하는게 보기 편하다
    c_report = classification_report(y_test, predict_result)
    print('classification_report:\n', c_report)

test_validation(rf_gs, X_test, y_test)

accuracy: 0.999
confusion_matrix:
 [[1932    0]
 [   2   66]]
classification_report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1932
           1       1.00      0.97      0.99        68

    accuracy                           1.00      2000
   macro avg       1.00      0.99      0.99      2000
weighted avg       1.00      1.00      1.00      2000



In [67]:
test_validation(rf_gs, X_test, y_test)

accuracy: 0.999
confusion_matrix:
 [[1932    0]
 [   2   66]]
classification_report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1932
           1       1.00      0.97      0.99        68

    accuracy                           1.00      2000
   macro avg       1.00      0.99      0.99      2000
weighted avg       1.00      1.00      1.00      2000



In [443]:
encoder = LabelEncoder()

In [ ]:
print(roc_auc_score(y_test, predict_result_2)) 